<a href="https://colab.research.google.com/github/amrrahman1/Sales_Pred_ML/blob/main/Sales_Pred_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
# mounting the drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [103]:
# importing panda
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor


In [104]:
# uploading the work sheet
filename = '/content/drive/MyDrive/Coding Dojo/sales_predictions.csv'

sales_predictions = pd.read_csv(filename)

In [105]:
# verifying data
sales_predictions.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [106]:
#1) How many rows and columns?
sales_predictions.info

<bound method DataFrame.info of      Item_Identifier  Item_Weight  ...        Outlet_Type  Item_Outlet_Sales
0              FDA15        9.300  ...  Supermarket Type1          3735.1380
1              DRC01        5.920  ...  Supermarket Type2           443.4228
2              FDN15       17.500  ...  Supermarket Type1          2097.2700
3              FDX07       19.200  ...      Grocery Store           732.3800
4              NCD19        8.930  ...  Supermarket Type1           994.7052
...              ...          ...  ...                ...                ...
8518           FDF22        6.865  ...  Supermarket Type1          2778.3834
8519           FDS36        8.380  ...  Supermarket Type1           549.2850
8520           NCJ29       10.600  ...  Supermarket Type1          1193.1136
8521           FDN46        7.210  ...  Supermarket Type2          1845.5976
8522           DRG01       14.800  ...  Supermarket Type1           765.6700

[8523 rows x 12 columns]>

In [107]:
# 2) What are the datatypes of each variable?
sales_predictions.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [108]:
## 3) Are there duplicates? If so, drop any duplicates.
# check for Duplicates :
sales_predictions.duplicated().any()

False

In [109]:
#4) Identify missing values.
sales_predictions.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [110]:
#7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent) 
sales_predictions[	"Item_Fat_Content"	].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [111]:
# fixing LF, low fat and reg
sales_predictions["Item_Fat_Content"]= sales_predictions["Item_Fat_Content"].replace('reg', "Regular")
sales_predictions["Item_Fat_Content"]= sales_predictions["Item_Fat_Content"].replace('LF', "Low Fat")
sales_predictions["Item_Fat_Content"]= sales_predictions["Item_Fat_Content"].replace('low fat',"Low Fat")


In [112]:
sales_predictions[	"Item_Fat_Content"	].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [113]:
# Identify the target (X) and features (y): Assign the "Item_Outlet_Sales" column as your target and the rest of the relevant variables as your features matrix.  
X = sales_predictions.drop(columns = 'Item_Outlet_Sales')
y = sales_predictions['Item_Outlet_Sales']

#Perform a train test split 
X_train,X_test, y_train, y_test = train_test_split(X,y, random_state =42)

In [114]:
# Selecting cat and num columns
num_selector = make_column_selector(dtype_include='number')
cat_selector = make_column_selector(dtype_include='object')

In [115]:
sales_predictions_num = sales_predictions[num_columns]
sales_predictions_num.loc[:, sales_predictions_num.isna().any()]

,Item_Weight
0,9.300
1,5.920
2,17.500
3,19.200
4,8.930
...,...
8518,6.865
8519,8.380
8520,10.600
8521,7.210


In [116]:
# Choosing median to fill in the missing data for the Item_weight column
median_imputer = SimpleImputer(strategy='median')

# Fitting the mean imputer on the X train
median_imputer.fit(X_train[num_columns])

SimpleImputer(strategy='median')

In [117]:
#transforming the numeric columns of both the training and testing sets.
X_train.loc[:, num_selector] = median_imputer.transform(X_train[num_selector])
X_test.loc[:, num_selector] = median_imputer.transform(X_test[num_selector])

In [118]:
# using the Frequant imputer on the catagory data to fill missing value.
freq_imputer = SimpleImputer(strategy='most_frequent')

# Fitting on X train.
freq_imputer.fit(X_train[cat_columns])

SimpleImputer(strategy='most_frequent')

In [119]:
# Transforming both the Xtrain and X test sets.
X_train.loc[:, cat_columns] = freq_imputer.transform(X_train[cat_columns])
X_test.loc[:, cat_columns] = freq_imputer.transform(X_test[cat_columns])

In [120]:
# Making sure there is no missing values
print(X_train.isna().sum().sum(), 'missing values')
X_train.info()

0 missing values
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6392 entries, 4776 to 7270
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6392 non-null   object 
 1   Item_Weight                6392 non-null   float64
 2   Item_Fat_Content           6392 non-null   object 
 3   Item_Visibility            6392 non-null   float64
 4   Item_Type                  6392 non-null   object 
 5   Item_MRP                   6392 non-null   float64
 6   Outlet_Identifier          6392 non-null   object 
 7   Outlet_Establishment_Year  6392 non-null   float64
 8   Outlet_Size                6392 non-null   object 
 9   Outlet_Location_Type       6392 non-null   object 
 10  Outlet_Type                6392 non-null   object 
dtypes: float64(4), object(7)
memory usage: 599.2+ KB


In [121]:
# Instintiate the OHE
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')


In [122]:
# Applying the encoder to the catagory data
ohe_tuple = (encoder, cat_selector)
column_trans = make_column_transformer(ohe_tuple,remainder='passthrough')

In [123]:
# Fitting the model on the training set.
lin_reg = LinearRegression()
pipe = make_pipeline(column_trans,lin_reg)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f9ea3916cd0>)])),
                ('linearregression', LinearRegression())])

In [124]:
def evaluate_model(y_true, y_pred):
  """takes two arrays, true labels and predicted labels, and prints
  MAE, MSE, RMSE, and R2 metrics"""
  mae = mean_absolute_error(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mean_squared_error(y_true, y_pred))
  r2 = r2_score(y_true, y_pred)

  print(f'scores: MAE: {mae}, \nMSE: {mse}, \nRMSE: {rmse}, \
  \nR2: {r2}')

In [125]:
# Evaluating the training and testing data
print('Training Scores')
evaluate_model(y_train, pipe.predict(X_train))

print('\n')

print('Testing Scores')
evaluate_model(y_test, pipe.predict(X_test))

Training Scores
scores: MAE: 735.647246970823, 
MSE: 971598.3192771976, 
RMSE: 985.6968698728822,   
R2: 0.6716969550207781


Testing Scores
scores: MAE: 996629998.7193755, 
MSE: 1.4286818007200278e+20, 
RMSE: 11952747804.249983,   
R2: -51783021449074.01


According to the r2 score of the testing and training data the model has a high variance (over fitted)

According to the RMSE score the model didnt perform on the training or the testing scores

*   List item
*   List item



In [126]:
# Intiating the deceision tree regressor.
dec_tree = DecisionTreeRegressor(random_state = 42)



In [127]:
# Fitting the model on the training data
pipe_tree = make_pipeline(column_trans, dec_tree)
pipe_tree.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7f9ea3916cd0>)])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [128]:
# Evaluating the Model.
print('Training Scores')
evaluate_model(y_train, pipe_tree.predict(X_train))

print('\n')

print('Testing Scores')
evaluate_model(y_test, pipe_tree.predict(X_test))

Training Scores
scores: MAE: 1.2005415435245748e-16, 
MSE: 2.4643264323299693e-29, 
RMSE: 4.96419825584149e-15,   
R2: 1.0


Testing Scores
scores: MAE: 990.1798845612387, 
MSE: 2114424.629630816, 
RMSE: 1454.1061273616917,   
R2: 0.23362013925393643


According to the r2 score the model performed only on the training data however the model didnt score well on the testing data which means that the model had a high variance

Based on the RMSE scores the model performed well on the training data however on the testing set it didnt perform well.